In [1]:
from my_package.summary_evalution import *
import pandas as pd

In [2]:
df = pd.read_csv(r"BBC_6Sent_.csv")

In [3]:
df.head()

,Original Article,Original Summary,"('Tm',)","('Lex',)","('Luhn',)","('Lsa',)","('Tr',)","('LDA',)","('Tm', 'Lex')","('Tm', 'Luhn')",...,"('Lex', 'Luhn', 'Tr', 'LDA')","('Lex', 'Lsa', 'Tr', 'LDA')","('Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr')","('Tm', 'Lex', 'Luhn', 'Lsa', 'LDA')","('Tm', 'Lex', 'Luhn', 'Tr', 'LDA')","('Tm', 'Lex', 'Lsa', 'Tr', 'LDA')","('Tm', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')"
0,Ad sales boost Time Warner profit\r\n\r\nQuart...,TimeWarner said fourth quarter sales rose 2% t...,The company said it was unable to estimate the...,"However, the company said AOL's underlying pro...","For the full-year, TimeWarner posted a profit ...",The company said it was unable to estimate the...,Time Warner's fourth quarter profits were slig...,The company said it was unable to estimate the...,The company said it was unable to estimate the...,"For the full-year, TimeWarner posted a profit ...",...,Ad sales boost Time Warner profit\r\n\r\nQuart...,Ad sales boost Time Warner profit\r\n\r\nQuart...,Ad sales boost Time Warner profit\r\n\r\nQuart...,Ad sales boost Time Warner profit\r\n\r\nQuart...,The company said it was unable to estimate the...,Ad sales boost Time Warner profit\r\n\r\nQuart...,The company said it was unable to estimate the...,Ad sales boost Time Warner profit\r\n\r\nQuart...,Ad sales boost Time Warner profit\r\n\r\nQuart...,Ad sales boost Time Warner profit\r\n\r\nQuart...
1,Dollar gains on Greenspan speech\r\n\r\nThe do...,The dollar has hit its highest level against t...,"""I think the chairman's taking a much more san...",Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,"On Friday, Federal Reserve chairman Mr Greensp...",Dollar gains on Greenspan speech\r\n\r\nThe do...,"On Friday, Federal Reserve chairman Mr Greensp...",Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,...,Dollar gains on Greenspan speech\r\n\r\nThe do...,"On Friday, Federal Reserve chairman Mr Greensp...",Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...,Dollar gains on Greenspan speech\r\n\r\nThe do...
2,Yukos unit buyer faces loan claim\r\n\r\nThe o...,Yukos' owner Menatep Group says it will ask Ro...,The owners of embattled Russian oil giant Yuko...,"""The pledged assets are with Rosneft, so it wi...",Yukos' owner Menatep Group says it will ask Ro...,Menatep Group's managing director Tim Osborne ...,The owners of embattled Russian oil giant Yuko...,Menatep Group's managing director Tim Osborne ...,State-owned Rosneft bought the Yugansk unit fo...,State-owned Rosneft bought the Yugansk unit fo...,...,State-owned Rosneft bought the Yugansk unit fo...,The owners of embattled Russian oil giant Yuko...,The owners of embattled Russian oil giant Yuko...,State-owned Rosneft bought the Yugansk unit fo...,State-owned Rosneft bought the Yugansk unit fo...,State-owned Rosneft bought the Yugansk unit fo...,The owners of embattled Russian oil giant Yuko...,The owners of embattled Russian oil giant Yuko...,State-owned Rosneft bought the Yugansk unit fo...,State-owned Rosneft bought the Yugansk unit fo...
3,High fuel prices hit BA's profits\r\n\r\nBriti...,"Rod Eddington, BA's chief executive, said the ...",Yet aviation analyst Mike Powell of Dresdner K...,High fuel prices hit BA's profits\r\n\r\nBriti...,To help offset the increased price of aviation...,Yet aviation analyst Mike Powell of Dresdner K...,"However, it said sales would be better than pr...",High fuel prices hit BA's profits\r\n\r\nBriti...,Yet aviation analyst Mike Powell of Dresdner K...,Yet aviation

In [8]:
key = df.columns[2]
summdf = pd.DataFrame([df[key]])
summdf = summdf.T

In [ ]:
output = rouge_scores_df(df,summdf)
output = df_avg_by_column(output)

In [ ]:
output.T

In [ ]:
pd.DataFrame(output).T

In [ ]:
pd.DataFrame(output).T

In [11]:
frames = []
algos = df.columns[2:]
for key in algos:
    summdf = pd.DataFrame([df[key]])
    summdf = summdf.T
    temp = calculate_rouge_scores_df(df,summdf)
    temp = calculate_average_by_column_df(temp)
    temp = pd.DataFrame(temp).T
    temp["Algorithm"] = key
    frames.append(temp)
out = pd.concat(frames)

100%|██████████| 2223/2223 [04:24<00:00,  8.42it/s]


In [12]:
cols = list(out.columns)
cols = [cols[-1]] + cols[:-1]
out = out[cols]

In [13]:
out = out.T
out.columns = out.iloc[0]
out = out[1:]
out

Algorithm,"('Tm',)","('Lex',)","('Luhn',)","('Lsa',)","('Tr',)","('LDA',)","('Tm', 'Lex')","('Tm', 'Luhn')","('Tm', 'Lsa')","('Tm', 'Tr')",...,"('Lex', 'Luhn', 'Tr', 'LDA')","('Lex', 'Lsa', 'Tr', 'LDA')","('Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr')","('Tm', 'Lex', 'Luhn', 'Lsa', 'LDA')","('Tm', 'Lex', 'Luhn', 'Tr', 'LDA')","('Tm', 'Lex', 'Lsa', 'Tr', 'LDA')","('Tm', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')","('Tm', 'Lex', 'Luhn', 'Lsa', 'Tr', 'LDA')"
rouge1_fmeasure,0.670829,0.667346,0.685291,0.638311,0.540117,0.638285,0.682031,0.700712,0.662431,0.626527,...,0.663066,0.63288,0.655018,0.678882,0.687134,0.679816,0.653726,0.669762,0.665921,0.675391
rouge1_precision,0.740551,0.73191,0.711519,0.727073,0.559545,0.728539,0.755674,0.761052,0.749863,0.678646,...,0.722612,0.712782,0.728625,0.753424,0.772502,0.748945,0.73736,0.748486,0.742789,0.755236
rouge1_recall,0.655214,0.656352,0.717991,0.602043,0.579555,0.600626,0.662642,0.697045,0.628895,0.628852,...,0.663387,0.605958,0.637044,0.659525,0.658042,0.668122,0.623717,0.646438,0.644895,0.650994
rouge2_fmeasure,0.583441,0.574351,0.604398,0.533102,0.414374,0.532056,0.597833,0.626698,0.569087,0.525308,...,0.577189,0.528436,0.563041,0.5959,0.606022,0.598454,0.557205,0.582609,0.577611,0.590412
rouge2_precision,0.648776,0.635567,0.631229,0.613984,0.43613,0.614004,0.666937,0.684333,0.649421,0.574397,...,0.633696,0.601748,0.6311,0.665522,0.685515,0.663389,0.633815,0.655266,0.64899,0.664439
rouge2_recall,0.56726,0.561288,0.631029,0.49859,0.439294,0.496416,0.57832,0.621404,0.537383,0.523998,...,0.574655,0.501835,0.545096,0.576751,0.578252,0.58606,0.528784,0.560254,0.556834,0.566934
rougeL_fmeasure,0.437776,0.431647,0.462845,0.404178,0.34484,0.406564,0.444736,0.470745,0.425227,0.400026,...,0.436535,0.40105,0.42765,0.448205,0.451509,0.449719,0.419817,0.438553,0.434888,0.443927
rougeL_precision,0.489775,0.479581,0.486568,0.46689,0.359724,0.470412,0.499147,0.518018,0.487552,0.437132,...,0.481687,0.45725,0.482749,0.50404,0.514781,0.502268,0.479853,0.496706,0.492412,0.503298
rougeL_recall,0.423121,0.420444,0.480221,0.377295,0.368293,0.378611,0.427769,0.463692,0.399812,0.398918,...,0.432484,0.380324,0.410819,0.430835,0.427962,0.437237,0.396374,0.418919,0.415955,0.42326
rougeLsum_fmeasure,0.59054,0.578593,0.602971,0.555277,0.462526,0.556491,0.599566,0.622646,0.579467,0.546782,...,0.581469,0.547787,0.576012,0.60069,0.607843,0.599256,0.572433,0.59187,0.5853,0.597396


In [16]:
out.to_csv("BBC_7Sent_Score.csv", index=False)

# run 2

In [17]:
df = pd.read_csv(r"BBC_6Sent_.csv")
frames = []
algos = df.columns[2:]
for key in algos:
    summdf = pd.DataFrame([df[key]])
    summdf = summdf.T
    temp = calculate_rouge_scores_df(df,summdf)
    temp = calculate_average_by_column_df(temp)
    temp = pd.DataFrame(temp).T
    temp["Algorithm"] = key
    frames.append(temp)
out = pd.concat(frames)
cols = list(out.columns)
cols = [cols[-1]] + cols[:-1]
out = out[cols]
out = out.T
out.columns = out.iloc[0]
out = out[1:]
out.to_csv("BBC_6Sent_Score.csv", index=False)

100%|██████████| 2224/2224 [04:28<00:00,  8.28it/s]


In [5]:
df = pd.read_csv(r"BBC_8Sent_.csv")
frames = []
algos = df.columns[2:]
for key in algos:
    summdf = pd.DataFrame([df[key]])
    summdf = summdf.T
    temp = calculate_rouge_scores_df(df,summdf)
    temp = calculate_average_by_column_df(temp)
    temp = pd.DataFrame(temp).T
    temp["Algorithm"] = key
    frames.append(temp)
out = pd.concat(frames)
cols = list(out.columns)
cols = [cols[-1]] + cols[:-1]
out = out[cols]
out = out.T
out.columns = out.iloc[0]
out = out[1:]
out.to_csv("BBC_8Sent_Score.csv", index=False)

100%|██████████| 2224/2224 [05:00<00:00,  7.41it/s]


In [4]:
df = pd.read_csv(r"BBC_1Dynamic-Sent_.csv")
frames = []
algos = df.columns[2:]
for key in algos:
    summdf = pd.DataFrame([df[key]])
    summdf = summdf.T
    temp = calculate_rouge_scores_df(df,summdf)
    temp = calculate_average_by_column_df(temp)
    temp = pd.DataFrame(temp).T
    temp["Algorithm"] = key
    frames.append(temp)
out = pd.concat(frames)
cols = list(out.columns)
cols = [cols[-1]] + cols[:-1]
out = out[cols]
out = out.T
out.columns = out.iloc[0]
out = out[1:]
out.to_csv("BBC_1Dynamic-Sent_Score.csv", index=False)

100%|██████████| 2224/2224 [01:55<00:00, 19.20it/s]
